In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
! pip install simpletransformers

     |████████████████████████████████| 221 kB 4.3 MB/s 
     |████████████████████████████████| 264 kB 33.2 MB/s 
     |████████████████████████████████| 8.0 MB 33.8 MB/s 
     |████████████████████████████████| 124 kB 47.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 1.2 MB 36.7 MB/s 
     |████████████████████████████████| 1.7 MB 39.4 MB/s 
     |████████████████████████████████| 2.8 MB 47.0 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 26.9 MB/s 
     |████████████████████████████████| 636 kB 37.0 MB/s 
     |████████████████████████████████| 170 kB 53.4 MB/s 
     |████████████████████████████████| 133 kB 48.0 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 119 kB 50.9 MB/s 
     |█████████████████

In [ ]:
import pandas as pd 
import numpy as np
import os
from google.colab import files
import matplotlib.pyplot as plt
import sklearn

#others
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from scipy.special import softmax
from sklearn.model_selection import train_test_split

#nn
from simpletransformers.classification import ClassificationModel
import collections

In [ ]:
url = 'https://raw.githubusercontent.com/JHDatz/UnicodeProjects/main/Twitter%20Project/Data/clean.csv'
df = pd.read_csv(url)
df.dropna(inplace = True)

In [ ]:
df

,injury_report,tweet,clean
0,0,I love the makeup of the Washington Football T...,I love makeup Washington Football Team The col...
1,0,@Darko12Donnie We’re a long way from that bein...,DarkoDonnie We long way issue
2,0,Guy got on the subway with an accordion and a ...,Guy got subway accordion tip jar Little know a...
3,0,Good catch by Danny Mac — Lars Nootbaar *is* w...,Good catch Danny Mac Lars Nootbaar stlcards Sa...
4,0,"At this point anything could happen, but the o...",At point anything could happen top options Red...
...,...,...,...
14207,1,He got hurt Tuesday night.,He got hurt Tuesday night
14208,1,He's a corner infielder coming off a hamstring...,Hes corner infielder coming hamstring injury S...
14209,1,Nats select contract of LHP Josh Rogers and mo...,Nats select contract LHP Josh Rogers move Kyle...
14210,1,"Cubs' Ross, Hoyer test positive for COVID-19, ...",Cubs Ross Hoyer test positive COVID feel fine ...


Sampling

In [ ]:
df_new1 = df[df['injury_report']==0]
df_new2 = df[df['injury_report']==1]

df_new1 = df_new1.sample(len(df_new2)*3, replace = True)

df = pd.concat([df_new1,df_new2])
df.reset_index(drop = True, inplace = True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['clean'], df['injury_report'], random_state = 91, stratify = df['injury_report'], test_size = 0.2)

train = pd.concat([X_train, y_train], axis = 1)
test = pd.concat([X_test, y_test], axis = 1)
y = test['injury_report'].tolist()

train = train[['clean','injury_report']]
test = test[['clean','injury_report']]
tst = test['clean'].tolist()

print(collections.Counter(train["injury_report"]))
print(collections.Counter(test["injury_report"]), end = "\n\n")


train.reset_index().to_csv("train 09-09.csv")
files.download('train 09-09.csv')
test.reset_index().to_csv("test 09-09.csv")
files.download('test 09-09.csv')

Counter({0: 8527, 1: 2842})
Counter({0: 2132, 1: 711})



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train

,clean,injury_report
4293,Promotion Olympics Futures Game JRODshow takin...,0
4317,Servais said use next couple days discuss star...,0
3074,Dodgers covering bases inning,0
2522,ableM How dare son,0
5092,Cubs crawlyscubs Yo Happy bday Crawly I would ...,0
...,...,...
12391,Dbacks mgr Torey Lovullo said imaging Madison ...,1
10698,Tyler ONeill hit mph fastball near right wrist...,1
11612,Bryant hamstring Báez thumb still lineup,1
771,ETD httpstcoAbbWLus,0


In [ ]:
test

,clean,injury_report
13332,Anthony DeSclafani arm fatigue,1
2712,They called Freddie second play advancing seco...,0
5173,José Álvarez ejected way field Going style,0
5856,Alex Wood strike bases loaded He ground though...,0
4027,Most Yankees nostalgia visits Minute Maid Park...,0
...,...,...
7909,CrawfordMILB Over,0
1329,MatthewHLeach KurkjianESPN Expand field add De...,0
6115,CraigHarmann brianmctaggart maybe tomorrowtoda...,0
879,When Wink IntentionalTalk know going laughs ht...,0


In [ ]:
def evaluations(model, model_name):
  result, model_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)
  print("Result: ")
  print(result, end = "\n\n")
  
  df_model_outputs = pd.DataFrame(model_outputs, columns = ['P0','P1'])
  df_model_outputs.to_csv(model_name +'_model_outputs.csv')
  files.download(model_name +'_model_outputs.csv')
  print(model_name + 'outputs downloaded', end = '\n\n')

  print(wrong_predictions, end = '\n\n')
  
  fn = []
  fp = []
  for i in range(len(model_outputs)):
    x = model_outputs[i];
    if x[1]>x[0] and y[i]==0:
      fp.append([i,tst[i]])
    elif x[0]>x[1] and y[i]==1:
      fn.append([i,tst[i]])
  print(fp)
  print(len(fp))
  print(fn)
  print(len(fn))
  
  df_fp = pd.DataFrame(fp, columns = ['index','fp tweet'])
  df_fp.to_csv(model_name + '_fp_tweets.csv')
  files.download(model_name + '_fp_tweets.csv')

  df_fn = pd.DataFrame(fn, columns = ['index','fn tweet'])
  df_fn.to_csv(model_name + '_fn_tweets.csv')
  files.download(model_name + '_fn_tweets.csv')

  print(model_name + ' FP and FN tweet dataframes downloaded', end = '\n\n')

  df_model_result = pd.DataFrame.from_dict(result, orient ='index')
  df_model_result.to_csv(model_name +'_model_result.csv')
  files.download(model_name +'_model_result.csv')
  print(model_name + 'result downloaded', end = '\n\n')

In [ ]:
def model_setup(arch, id):

  #initialise
  model =  ClassificationModel(arch, id, num_labels=2, use_cuda=True, args={
        'reprocess_input_data': True,
        'use_cached_eval_features': False,
        'overwrite_output_dir': True,
        'num_train_epochs': 5,
        'use_early_stopping': True,
        'early_stopping_delta': 0.01,
        'early_stopping_metric': "mcc",
        'early_stopping_metric_minimize': False,
        'early_stopping_patience': 5,
        'evaluate_during_training_steps': 1000
      })
  
  # print config
  print(model.config, end = '\n\n')

  #train
  print('Training starts')
  model.train_model(train)
  evaluations(model, id + '-5-epochs')
  
  return model

### DISTILBERT MODEL

In [ ]:
bert = model_setup('distilbert', 'distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.10.0",
  "vocab_size": 30522
}


Training starts


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:616: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/11369 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

...

<IPython.core.display.Javascript object>

distilbert-base-uncased-5-epochs FP and FN tweet dataframes downloaded



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

distilbert-base-uncased-5-epochsresult downloaded



In [ ]:
df

,injury_report,tweet,clean
0,0,I love the makeup of the Washington Football T...,I love makeup Washington Football Team The col...
1,0,@Darko12Donnie We’re a long way from that bein...,DarkoDonnie We long way issue
2,0,Guy got on the subway with an accordion and a ...,Guy got subway accordion tip jar Little know a...
3,0,Good catch by Danny Mac — Lars Nootbaar *is* w...,Good catch Danny Mac Lars Nootbaar stlcards Sa...
4,0,"At this point anything could happen, but the o...",At point anything could happen top options Red...
...,...,...,...
14207,1,He got hurt Tuesday night.,He got hurt Tuesday night
14208,1,He's a corner infielder coming off a hamstring...,Hes corner infielder coming hamstring injury S...
14209,1,Nats select contract of LHP Josh Rogers and mo...,Nats select contract LHP Josh Rogers move Kyle...
14210,1,"Cubs' Ross, Hoyer test positive for COVID-19, ...",Cubs Ross Hoyer test positive COVID feel fine ...


### XLNET CONFIG

In [ ]:
bert2 = model_setup('xlnet', 'xlnet-base-cased')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "transformers_version": "4.10.0",
  "untie_r": true,
  "use_mems_eval": true,
  "use_mems_train": false,
  "vocab_size": 32000
}


Training starts


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:616: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/11369 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/1422 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarnin

KeyboardInterrupt: ignored

### ROBERTA

In [ ]:
bert3 = model_setup('roberta', 'roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_p

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

roberta-base-5-epochs FP and FN tweet dataframes downloaded



In [ ]:
result

NameError: ignored

### XLM-ROBERTA

In [ ]:
bert4 = model_setup('xlmroberta', 'xlm-roberta-base')

### Electra Model

In [ ]:
bert5 = model_setup('electra', 'google/electra-base-discriminator')

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.w

ElectraConfig {
  "_name_or_path": "google/electra-base-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "vocab_size": 30522
}


Training starts


RuntimeError: ignored

### distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
bert6 = model_setup('distilbert', 'distilbert-base-uncased-finetuned-sst-2-english')